# Combining behavioral and neurophysiological data


The goal of this workshop is to introduce a way of combining neurophysiological data and behavioral observations, on recordings of human EEG in a discrimination task. 

## 1. Description of the experiment and data

In this task, participants were presented at each trial either with an image of a face or a sound, and had to report what they have perceived through key presses (left arrow=sound, right arrow=face) as fast and accuately as possible. A timeout on response time would occur two seconds after stimulus presentation. 

The experiment is split into two parts:
- a calibration phase, where the stimuli were preceded by a predictive cue, i.e. a stimulus to which participants did not have to response, but that predicted with a 80% accuracy the stimulus that is coming next
- a test phase, whose sequence does not include cues.

While performing this task, their EEG activity was recorded using 32 channels, along with their response class and time.

## 2. Import and visualize data
In this first part we look at raw data and see how we can improve it.

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from utils import import_raw,get_epochs,plot_ERP

In [ ]:
# import raw data
subject='756'
datapath='D:\\PhD\\my_data\\{}\\{}_calib_000{}'.format(subject,subject,{})
datapath_test='D:\\PhD\\my_data\\{}\\{}_test_000{}'.format(subject,subject,{})
nblocks=4
nblocks_test=3

In [ ]:
eegdata=import_raw(datapath,nblocks)

# plot electrode location
eegdata.plot_sensors(show_names=True)

# epoch data
events=[11,12]
tmin=-1
tmax=0.5
epochs=get_epochs(eegdata, events,tmin=tmin, tmax=tmax)

# plot ERP
plot_ERP(epochs,['11','12'])

We can see that the data is quite messy, as it is prone to artifacts. In fact, with EEG, any small movement can appear in the signal, from blinking, to sweating, to clenching the jaw. It is necessary to remove them. Run the code below to see how this is improved.

In [ ]:
#filter data out
fmin=None
fmax=30
eegdata_filtered=eegdata.filter(fmin, fmax, fir_design='firwin')

In [ ]:
#reject outliers that cannot be filtered
#first, we can check if some electrodes have too high impedances
from utils import get_bad_impedance, get_bad_channels_trials

get_bad_impedance(eegdata_filtered)
print(eegdata_filtered.info['bads'])
#then we reject epochs and electrodes that are above a certain threshold 
rejt,rejc=get_bad_channels_trials(eegdata_filtered,events)
eegdata_filtered.info['bads'].extend(rejc)
eegdata_filtered = eegdata_filtered.copy().interpolate_bads(reset_bads=False)
print(eegdata_filtered.info['bads'])

#plot new ERP
clean_epochs=get_epochs(eegdata_filtered, events,tmin=tmin, tmax=tmax)
clean_epochs.drop(rejt)
plot_ERP(clean_epochs,['11','12'])

This is already much better. Pre-processing is a whole topic to itself, so we will not go into detail in that part, you have the whole master to learn it.

## 3. Look at the behavior
We are not only interested in the EEG, but also in how the participants behaved and in particular how fast they responsed. In the following section, we extract the response times and look at their distribution.

In [ ]:
from utils import get_RT, get_labels

# extract response times
responses=[1001,1002,1003]
RT=get_RT(eegdata_filtered, events, responses, rejt)

# extract labels
labels=get_labels(clean_epochs)

# plot histograms
plt.figure()
plt.hist(RT[labels==11],alpha=0.2,label='Visual, mean={:.0f} ms'.format(np.mean(RT[labels==11])))
plt.hist(RT[labels==12],alpha=0.2,label='Auditory, mean={:.0f} ms'.format(np.mean(RT[labels==12])))
plt.title('Histogramm of response times, mean={:.0f} ms'.format(np.mean(RT)))
plt.legend()

We can also gain insight from the errors that people have made.

In [1]:
# count number of errors
from utils import get_errors

errors=get_errors(eegdata_filtered, events,[1001,1002,1003],rejt)
print('This participant committed {} errors'.format(np.sum(errors)))

## 4. Inter-trial phase coherence (ITPC)
Also called intertrial phase clustering, this technique presents a global view on the phases at a specific time-point in all the trials. In particular, it tells you how consistent the phase at that time point is in all trials.

The ERP, that we have seen before, is extracted through averaging all trials. Averaging is very useful to observe global tendecies of the signal amplitude. It would be nice to observe the phase in the same manner. However, it is not possible to average phases because of their circularity: $0$ and $2 \pi$ are the same angle, but they are numerically very far! Simple averaging is not possible, but what we can do is convert the angles in their exponential notation.

## 5. Combining ITPC and other quantitative data: wITPC
From the ITPC exists a derived quantity called weighted ITPC, which allows to put in relation the EEG and another quantity: response times, pupil diameter, heart rate, etc. It is particularly useful for putting in relation behavior and phases, and is an elegant alternative to correlations.